In [1]:
import sys
# !{sys.executable} -m pip install geocoder
# !{sys.executable} -m pip install geopy
# !{sys.executable} -m pip install wget

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#Download and explore the dataset
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded')

Data downloaded


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neigbourhoods_data = newyork_data['features']

In [5]:
neigbourhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [6]:
#transform data into pandas dataframe

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

neigbourhoods = pd.DataFrame(columns=column_names)

In [7]:
neigbourhoods

,Borough,Neighborhood,Latitude,Longitude


In [8]:
# Next we loop over neighneigbourhoods_data and append relevant data
#to neigbors

for data in neigbourhoods_data:
    borough = neigbourhoods_name = data['properties']['borough']
    neigbourhoods_name = data['properties']['name']
    
    neigbourhoods_latlon = data['geometry']['coordinates']
    neigbourhoods_lat = neigbourhoods_latlon[1]
    neigbourhoods_lon = neigbourhoods_latlon[0]
    neigbourhoods = neigbourhoods.append({
        'Borough': borough,
        'Neighborhood': neigbourhoods_name,
        'Latitude': neigbourhoods_lat,
        'Longitude': neigbourhoods_lon,
    }, ignore_index=True)

In [9]:
neigbourhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neigbourhoods['Borough'].unique()),
        neigbourhoods.shape[0]
    )
)


The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
#Next we use geopy library to get the latitude and longitude values of the New york city

In [12]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent='explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of New york city are {}, {}'.format(latitude, longitude))

The geographical coordinate of New york city are 40.7127281, -74.0060152


In [13]:
#Next lets create a map of new york with the defined neigborbhoods

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

#add neigborhoods marker
for lat, lon, borough, neigbourhood in zip(neigbourhoods['Latitude'], neigbourhoods['Longitude'], neigbourhoods['Borough'], neigbourhoods['Neighborhood']):
    label = '{}, {}'.format(neigbourhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity='0.7',
        parse_html=False
    ).add_to(map_newyork)
map_newyork

In [14]:
#lets visualize the manhattan using the longitute and latitude
manhattan = neigbourhoods[neigbourhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
address = 'Manhattan New york'

# geolocator = Nominatim(user_agent="ny_explorer")
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude

latitude = 40.7831
longitude = -73.9712
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7831, -73.9712.


In [16]:
#Next lets create the manhattan map using the longitude and longitude

map_Man = folium.Map(location = [latitude, longitude], zoom_start=11)

#add markers
for lat, lng, label in zip(manhattan['Latitude'], manhattan['Longitude'], manhattan['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        color='blue',
        radius = 4,
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_Man)
map_Man

## Lets start utilizing foursquare API to explore the neigbhoods and segment them

In [17]:
#Define foursquare credentials and Version
Client_id = 'GGSRESGUNZWU4IWX5LPPAR12FCPPMCNJNMCNQWAFGVIOQFGN'
Client_secret = 'Y2AP3Y1V1R5C3UUWF2QITSMS4QSLLHQ0UIFGLK3VPBWTTVUK'
Version = '20200202'
LIMIT = 100
radius=500
print('your credentials: ')
print(Client_id)
print(Client_secret)

your credentials: 
GGSRESGUNZWU4IWX5LPPAR12FCPPMCNJNMCNQWAFGVIOQFGN
Y2AP3Y1V1R5C3UUWF2QITSMS4QSLLHQ0UIFGLK3VPBWTTVUK


In [18]:
#explore the neighborhood in our dataset
manhattan.loc[0,'Neighborhood']

'Marble Hill'

In [21]:
#Lets get the neighborhood latitude and longitude
man_neigbourhood_lat = manhattan.loc[0, 'Latitude']
man_neigbourhood_lng = manhattan.loc[0, 'Longitude']

man_neigbourhood_name = manhattan.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(man_neigbourhood_name, 
                                                               man_neigbourhood_lat, 
                                                               man_neigbourhood_lng))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [22]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(Client_id, Client_secret, latitude, longitude, Version, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=GGSRESGUNZWU4IWX5LPPAR12FCPPMCNJNMCNQWAFGVIOQFGN&client_secret=Y2AP3Y1V1R5C3UUWF2QITSMS4QSLLHQ0UIFGLK3VPBWTTVUK&ll=40.7831,-73.9712&v=20200202&radius=500&limit=100'

In [23]:
#Send get request
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5e84e8013907e7001b6f94d1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Upper West Side',
  'headerFullLocation': 'Upper West Side, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 98,
  'suggestedBounds': {'ne': {'lat': 40.7876000045, 'lng': -73.96526804660982},
   'sw': {'lat': 40.778599995499995, 'lng': -73.97713195339017}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a5fe58ef964a5207fc01fe3',
       'name': 'Rose Center for Earth and Space',
       'location': {'address': '200 Central Park W',
        'crossStreet': 'btwn W 77th & W 81st St',
        'lat': 40.781740

In [24]:
#Fuction to extract the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
#Lets clean the json file and structure it in a dataFrame
venues = result['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

<ipython-input-25-7500088e92e4>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


In [26]:
nearby_venues.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id,venue.location.neighborhood,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name
0,e-0-4a5fe58ef964a5207fc01fe3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4a5fe58ef964a5207fc01fe3,Rose Center for Earth and Space,200 Central Park W,btwn W 77th & W 81st St,40.781741,-73.973127,"[{'label': 'display', 'lat': 40.78174080376753...",221,10024,US,New York,NY,United States,"[200 Central Park W (btwn W 77th & W 81st St),...","[{'id': '4bf58dd8d48988d192941735', 'name': 'P...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-4b9af865f964a520fde735e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b9af865f964a520fde735e3,Hayden Planetarium,200 Central Park W,inside American Museum of Natural History,40.781718,-73.973239,"[{'label': 'display', 'lat': 40.78171789252808...",230,10024,US,New York,NY,United States,[200 Central Park W (inside American Museum of...,"[{'id': '4bf58dd8d48988d192941735', 'name': 'P...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,e-0-4c3f76b63735be9abe6a15a4-2,0,"[{'summary': 'This spot is popular', 'type': '...",4c3f76b63735be9abe6a15a4,Central Park - Mariners' Gate,West 85th St,Central Park West,40.784668,-73.969746,"[{'label': 'display', 'lat': 40.78466832421952...",213,10024,US,New York,NY,United States,"[West 85th St (Central Park West), New York, N...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e-0-4bddeb62ffdec92800f5e6a1-3,0,"[{'summary': 'This spot is popular', 'type': '...",4bddeb62ffdec92800f5e6a1,Summit Rock,NaN,just south of 85th street entrance at CPW,40.783461,-73.969979,"[{'label': 'display', 'lat': 40.78346109631921...",110,10024,US,New York,NY,United States,"[New York, NY 10024, United States]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e-0-5476852d498ee22f104ba7b8-4,0,"[{'summary': 'This spot is popular', 'type': '...",5476852d498ee22f104ba7b8,Book Culture,450 Columbus Ave,btwn W 81st & W 82nd St,40.783719,-73.974288,"[{'label': 'display', 'lat': 40.7837186733246,...",269,10024,US,New York,NY,United States,"[450 Columbus Ave (btwn W 81st & W 82nd St), N...","[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",0,[],110914052,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

#filtered_columns
nearby_venues = nearby_venues.loc[:, filtered_columns]
#nearby_venues = nearby_venues[filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean the columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rose Center for Earth and Space,Planetarium,40.781741,-73.973127
1,Hayden Planetarium,Planetarium,40.781718,-73.973239
2,Central Park - Mariners' Gate,Park,40.784668,-73.969746
3,Summit Rock,Park,40.783461,-73.969979
4,Book Culture,Bookstore,40.783719,-73.974288


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

98 venues were returned by Foursquare.


## Explore Neighborhood in Manhattan

In [29]:
#Lets create a function that repeats the same process to all neighborhood in manhattan

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_id,
            Client_secret,
            Version,
            lat, 
            lng, 
            radius, 
            LIMIT)
        #Make get request
        result = requests.get(url).json()["response"]["groups"][0]["items"]
        
        #return only relevant information for each nearby venue
        
        venues_list.append([(name, lat, lng, 
                           v['venue']['name'],
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in result])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_id, 
            Client_secret, 
            Version, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
manhattan_venues = getNearbyVenues(names=manhattan['Neighborhood'],
                                   latitudes=manhattan['Latitude'],
                                   longitudes=manhattan['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [32]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [33]:
manhattan_venues.shape

(3293, 7)

## Analyze Each Neighborhood

In [34]:
#one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

#add neigborhood colum back to dataframe
manhattan_onehot['Neigborhood'] = manhattan_venues['Neighborhood']

In [35]:
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

In [36]:
manhattan_onehot.shape

(3293, 334)

In [37]:
## Next lets group rows by neighborhood and by taking the mean of the frequency of occurency of each category

manhattan_group = manhattan_onehot.groupby('Neigborhood').mean().reset_index()
manhattan_group.head()

,Neigborhood,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,

In [38]:
#Next print each neighborhood along with the top5 most common venues
num_top_venues = 5

for hood in manhattan_group['Neigborhood']:
    print("------"+hood+"-------")
    temp=manhattan_group[manhattan_group['Neigborhood'] == hood].T.reset_index()
    temp.columns = ["venue", "freq"]
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

------Battery Park City-------
           venue  freq
0           Park  0.07
1          Hotel  0.06
2    Coffee Shop  0.06
3  Boat or Ferry  0.03
4      Wine Shop  0.03


------Carnegie Hill-------
                  venue  freq
0           Coffee Shop  0.06
1           Pizza Place  0.06
2                  Café  0.05
3  Gym / Fitness Center  0.03
4        Cosmetics Shop  0.03


------Central Harlem-------
                 venue  freq
0   African Restaurant  0.07
1        Deli / Bodega  0.05
2  American Restaurant  0.05
3                  Bar  0.05
4   Seafood Restaurant  0.05


------Chelsea-------
                venue  freq
0         Coffee Shop  0.07
1              Bakery  0.06
2  Italian Restaurant  0.04
3      Ice Cream Shop  0.03
4               Hotel  0.03


------Chinatown-------
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.05
2  American Restaurant  0.04
3                  Spa  0.03
4         Optical Shop  0.03


------Civic Center-------


In [39]:
def return_most_common_values(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0: num_top_venues]

In [42]:
columns = ['Neighborhood']
num_top_venues = 10
indicators = ['std', 'nd', 'rd']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_group['Neigborhood']

for ind in np.arange(manhattan_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_values(manhattan_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1std Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Wine Shop,Boat or Ferry,Memorial Site,Pizza Place,Italian Restaurant,BBQ Joint
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Bakery,Gym / Fitness Center,Gym,Japanese Restaurant,French Restaurant,Cosmetics Shop,Bookstore
2,Central Harlem,African Restaurant,Deli / Bodega,American Restaurant,Bar,French Restaurant,Seafood Restaurant,Chinese Restaurant,Cafeteria,Boutique,Beer Bar
3,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Hotel,American Restaurant,Ice Cream Shop,Nightclub,Bookstore,Market,French Restaurant
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Optical Shop,Bakery,Dessert Shop,Vietnamese Restaurant,Salon / Barbershop,Spa,Hotpot Restaurant


## Cluster Neighborhoods

In [46]:
k = 5

manhattan_group_clust = manhattan_group.drop('Neigborhood', 1)

#run k-means clustering
kmeans = KMeans (n_clusters=k, random_state=0).fit(manhattan_group_clust)

#Check cluster labels generated for eavh row in dataframe
kmeans.labels_[0:10]

array([1, 1, 3, 1, 1, 1, 1, 2, 1, 1], dtype=int32)

In [47]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan

#merge manhattan grouped data with manhattan to add lat and lng
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1std Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Pharmacy,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Optical Shop,Bakery,Dessert Shop,Vietnamese Restaurant,Salon / Barbershop,Spa,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Grocery Store,Bakery,Chinese Restaurant,Mobile Phone Shop,Latin American Restaurant,Bank,Sandwich Place,Tapas Restaurant,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,2,Café,Mexican Restaurant,Pizza Place,Lounge,Restaurant,Chinese Restaurant,Frozen Yogurt Shop,American Restaurant,Park,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Yoga Studio,Sandwich Place,Sushi Restaurant,Bakery,Caribbean Restaurant


In [50]:
#Finally lets visualize on map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for the clusters
x = np.arange(k)
ys=[i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
#Examine Clusters

In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1std Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Park,Gas Station,Gym / Fitness Center,Baseball Field,German Restaurant,Harbor / Marina,Cocktail Bar,Coffee Shop,Bistro


In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1std Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Optical Shop,Bakery,Dessert Shop,Vietnamese Restaurant,Salon / Barbershop,Spa,Hotpot Restaurant
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Exhibit,Juice Bar,Gym / Fitness Center,Art Gallery,French Restaurant,Yoga Studio,Pizza Place
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Bar,Deli / Bodega,Pizza Place,Mexican Restaurant,Japanese Restaurant,Diner,Sushi Restaurant
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Cocktail Bar,Burger Joint,Café,Gym,Gym / Fitness Center,Playground
12,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Wine Bar,Mediterranean Restaurant,Indian Restaurant,Bakery,Café,Ice Cream Shop,Yoga Studio
13,Lincoln Square,Italian Restaurant,Plaza,Theater,Café,Performing Arts Venue,American Restaurant,Concert Hall,Indie Movie Theater,Gym / Fitness Center,Wine Shop
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,American Restaurant,Spa,Coffee Shop,Sandwich Place,Wine Shop,Gym,Hotel
15,Midtown,Hotel,Steakhouse,Coffee Shop,Clothing Store,Bookstore,Bakery,French Restaurant,Café,Theater,Sporting Goods Shop
16,Murray Hill,Coffee Shop,Sandwich Place,American Restaurant,Japanese Restaurant,Hotel,Gym / Fitness Center,Italian Restaurant,Mediterranean Restaurant,Cocktail Bar,Burger Joint
17,Chelsea,Coffee Shop,Bakery,Italian Restaurant,Hotel,American Restaurant,Ice Cream Shop,Nightclub,Bookstore,Market,French Restaurant
